# Basics of Expression at M0

This notebook walks through the basics of executing Expressions at M0 after they are generated from M1. 

In [ ]:
from pathlib import Path

import pymbe.api as pm

from pymbe.client import SysML2Client
from pymbe.graph.calc_lpg import CalculationGroup
from pymbe.graph.lpg import SysML2LabeledPropertyGraph
from pymbe.interpretation.calc_dependencies import (
    generate_execution_order,
    generate_parameter_signature_map,
)
from pymbe.interpretation.interp_playbooks import (
    build_expression_sequence_templates,
    build_sequence_templates,
    random_generator_playbook,
)
from pymbe.interpretation.results import (
    pprint_calc_steps,
    pprint_double_id_list,
)
from pymbe.label import get_label_for_id
from pymbe.local.stablization import build_stable_id_lookups

In [ ]:
parts_client = SysML2Client()

simple_parts_file = Path(pm.__file__).parent / "../../tests/fixtures/Simple Expressions.json"

parts_client._load_from_file(simple_parts_file)

parts_lpg = SysML2LabeledPropertyGraph()
parts_lpg.model = parts_client.model

SIMPLE_MODEL = "Model::Simple Parts Model::"

[id_to_parts_name_lookup, parts_name_to_id_lookup] = build_stable_id_lookups(parts_lpg)

parts_lpg.model.max_multiplicity = 10

## Simple Expressions Example

### Calculation Order

Once the M1 model has been studied, an order of computation can be generated where knowns propagate to unknowns.

Literal expressions and Feature Reference Expressions should be run first, since there are either no inputs (literal expressions) or the inputs are pre-determined (feature reference expressions).

In [ ]:
calc_order = generate_execution_order(parts_lpg)
calc_order[0]

In [ ]:
parts_lpg.model.elements[calc_order[0][0]].qualifiedName

In [ ]:
filtered_order = [
    filtered_entry for filtered_entry in calc_order
    if parts_lpg.model.elements[filtered_entry[0]].qualifiedName.startswith("'Simple Expressions'")
]

In [ ]:
calculation_steps = pprint_calc_steps(
    filtered_order,
    parts_lpg.model,
    generate_parameter_signature_map(parts_lpg.model, calc_order),
)

In [ ]:
len(calculation_steps)

### Full Order

The full execution order is shown below.

In [ ]:
calculation_steps

### Selection Queries

The selection queries gather elements from the M0 execution and assign them to the results of Feature Reference Expressions.

In [ ]:
[sq for sq in calculation_steps if sq[2] == 'SelectionQuery']

### Outputs

The output steps take the body of evaluations and mark the sending of results to the result parameters.

In [ ]:
[sq for sq in calculation_steps if sq[2] == 'Output']

### Inputs

The input parameters passing values into the evaluations are shown below.

In [ ]:
[sq for sq in calculation_steps if sq[2] == 'Input']

### Assignments

The assignments have the mapping of result parameters from one set of evaluations to the input parameters to the next.

In [ ]:
[sq for sq in calculation_steps if sq[2] == 'Assignment']

### ValueBinding

The results of calculations applied to attribute usages are the final steps of the computations. This is also where the pieces of an expression tree are assembled and ready for binding to the value of the attribute.

In [ ]:
[sq for sq in calculation_steps if sq[2] == "ValueBinding"]

In [ ]:
value_binding_sequences = [sq for sq in filtered_order if sq[2] == "ValueBinding"]
value_binding_sequences

In [ ]:
feature_sequences = build_sequence_templates(lpg=parts_lpg)
pprint_double_id_list(feature_sequences, parts_lpg.model)

In [ ]:
feature_sequences = build_sequence_templates(lpg=parts_lpg)
simple_expressions = parts_lpg.model.ownedElement["Simple Expressions"]

In [ ]:
m0_interpretation = random_generator_playbook(
    parts_lpg,
    {},
    [simple_expressions],
)

See what M1 items in the calculation pairings have M0 interpretations.

In [ ]:
[
    [sq[0] in m0_interpretation, sq[1] in m0_interpretation]
    for sq in value_binding_sequences
]

In [ ]:
m0_interpretation

In [ ]:
[
     [id_to_parts_name_lookup[sq[0]], id_to_parts_name_lookup[sq[1]]]
     for sq in value_binding_sequences
]

In [ ]:
[
     [parts_lpg.model.elements[sq[0]].type, parts_lpg.model.elements[sq[1]].type]
     for sq in value_binding_sequences
]

In [ ]:
all_expr_sequences = build_expression_sequence_templates(lpg=parts_lpg)
expr_sequences = [
    seq for seq in all_expr_sequences
    if parts_lpg.model.elements[seq[-1]].owning_package in [simple_expressions]
]
[[id_to_parts_name_lookup[item] for item in seq] for seq in expr_sequences]

In [ ]:
cg = CalculationGroup(parts_lpg.get_projection("Expression Inferred"), m0_interpretation, filtered_order)
cg.solve_graph(parts_lpg)

In [ ]:
[
     (m0_interpretation[sq[0]], m0_interpretation[sq[1]], get_label_for_id(sq[1], parts_lpg.model))
     for sq in value_binding_sequences
]